# Dataset

## Extract text from PDF

In [1]:
import os
import re
import json
import pandas as pd
import spacy
from PyPDF2 import PdfReader
import pdfplumber
from pathlib import Path
import logging
from typing import Dict, List, Any, Optional, Tuple, Set

In [2]:
nlp = spacy.load("en_core_web_lg")

## Get Data

In [3]:
pdf_path = "reports/viewreport.pdf"

### Read PDF

In [4]:
with open("reports/viewreport.pdf","rb") as f:
    reader = PdfReader(f)
    text2 = ""
    for page in reader.pages:
        text2 += page.extract_text() + "\n"
print(text2[:400])


District 2, 25 - WINDHAM County
Owner: 3 - Town or Township Highway Agency
Town: 262 - WINDHAM
Maintenance Responsibility: 3 - Town or Township Highway Agency
Team Lead:
 Jeremy Spooner,  
Inspection Date:
 07/05/2024
Structure 
#00010 /  
(Routine)
Route 
FAS 0123 /  
TR 02  FAS 123 over MID. BR WILLIAMS RIVER

© OpenStreetMap contributors
Location: 0.3 MI S JCT. VT.11
43.21230, -72.73812
Team Le


### Try fitz

In [5]:
import fitz  # PyMuPDF


document = fitz.open(pdf_path)
    
    # Variable to store the text
text3 = ""
    
    # Iterate through each page
for page_num in range(len(document)):
     # Get the page
    page = document.load_page(page_num)
        
        # Extract text from the page
    page_text = page.get_text()
        
        # Add the page text to our text variable
    text3 += page_text
    
    # Close the document
document.close()
    
text3

'District 2, 25 - WINDHAM County\nOwner: 3 - Town or Township Highway Agency\nTown: 262 - WINDHAM\nMaintenance Responsibility: 3 - Town or Township Highway Agency\nTeam Lead: Jeremy Spooner,  Inspection Date: 07/05/2024\nStructure #00010 /  (Routine)\nRoute FAS 0123 /  \nTR 02  FAS 123 over MID. BR WILLIAMS RIVER\n© OpenStreetMap contributors\nLocation: 0.3 MI S JCT. VT.11\n43.21230, -72.73812\nTeam Lead: Jeremy Spooner,  Inspection Date: 07/05/2024\nStructure #00010 /  (Routine)\nRoute FAS 0123 /  \nTR 02  FAS 123 over MID. BR WILLIAMS RIVER\nIDENTIFICATION\n(1) State Names\n50 - Vermont\n(8) Structure Number\n200123001013232\n(5) Inventory Route\n1\n(2) Highway Agency District\n2 - District 2\n(3) County Code\n25 - WINDHAM\n(4) Place Code\n84850\n(6) Features Intersected\nMID. BR WILLIAMS RIVER\n(7) Facility Carried\nTR 02  FAS 123\n(9) Location\n0.3 MI S JCT. VT.11\n(11) Mile Point\n0 mi\n(12) Base Highway Network\nNo\n(13) LRS Inventory Rte & Subrte\n(16) Latitude\n43.2123\n(17) Lo

In [6]:
doc3 = nlp(text3)
doc3

District 2, 25 - WINDHAM County
Owner: 3 - Town or Township Highway Agency
Town: 262 - WINDHAM
Maintenance Responsibility: 3 - Town or Township Highway Agency
Team Lead: Jeremy Spooner,  Inspection Date: 07/05/2024
Structure #00010 /  (Routine)
Route FAS 0123 /  
TR 02  FAS 123 over MID. BR WILLIAMS RIVER
© OpenStreetMap contributors
Location: 0.3 MI S JCT. VT.11
43.21230, -72.73812
Team Lead: Jeremy Spooner,  Inspection Date: 07/05/2024
Structure #00010 /  (Routine)
Route FAS 0123 /  
TR 02  FAS 123 over MID. BR WILLIAMS RIVER
IDENTIFICATION
(1) State Names
50 - Vermont
(8) Structure Number
200123001013232
(5) Inventory Route
1
(2) Highway Agency District
2 - District 2
(3) County Code
25 - WINDHAM
(4) Place Code
84850
(6) Features Intersected
MID. BR WILLIAMS RIVER
(7) Facility Carried
TR 02  FAS 123
(9) Location
0.3 MI S JCT. VT.11
(11) Mile Point
0 mi
(12) Base Highway Network
No
(13) LRS Inventory Rte & Subrte
(16) Latitude
43.2123
(17) Longitude
-72.7381222222222
(98) Border Brid

### Try another

In [7]:
from pdfminer.high_level import extract_text

text4 = extract_text(pdf_path)
doc4 = nlp(text4)
doc4

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Route FAS 0123 /  

Structure #00010 /  (Routine)

TR 02  FAS 123 over MID. BR WILLIAMS RIVER

Team Lead: Jeremy Spooner,  Inspection Date: 07/05/2024

Town: 262 - WINDHAM

District 2, 25 - WINDHAM County

Owner: 3 - Town or Township Highway Agency

Maintenance Responsibility: 3 - Town or Township Highway Agency


Route FAS 0123 /  

Structure #00010 /  (Routine)

TR 02  FAS 123 over MID. BR WILLIAMS RIVER

Team Lead: Jeremy Spooner,  Inspection Date: 07/05/2024

Location: 0.3 MI S JCT. VT.11

43.21230, -72.73812

© OpenStreetMap contributors


Route FAS 0123 /  

Structure #00010 /  (Routine)

TR 02  FAS 123 over MID. BR WILLIAMS RIVER

Team Lead: Jeremy Spooner,  Inspection Date: 07/05/2024

IDENTIFICATION

CLASSIFICATION

(1) State Names
(8) Structure Number
(5) Inventory Route
(2) Highway Agency District
(3) County Code
(4) Place Code
(6) Features Intersected
(7) Facility Carried
(9) Location
(11) Mile Point
(12) Base Highway Network
(13) LRS Inventory Rte & Subrte
(16) Latitude
(1

### Extract text better

In [8]:
text_with_tables = ""
tables = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        # Extract tables first
        table_per_page = page.extract_tables()
        if(len(table_per_page) > 0):
            tables.append(table_per_page)
        # # Format tables as text
        for table in tables:
            for row in table:
                for a_row in row:
                    # print(a_row)
        #         # Filter None values and join cells with tabs
            #     print(row)
                    row_text = "\t".join([cell[0] if cell else "" for cell in a_row])
                    text_with_tables += row_text + "\n"
                text_with_tables += "\n"  # Extra line between tables
            
        #     # Extract the remaining text
        text = page.extract_text()
        if text:
            text_with_tables += text + "\n\n"
text_with_tables

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

'Route FAS 0123 /\nStructure #00010 / (Routine)\nTR 02 FAS 123 over MID. BR WILLIAMS RIVER\nTeam Lead: Jeremy Spooner, Inspection Date: 07/05/2024\nTown: 262 - WINDHAM\nDistrict 2, 25 - WINDHAM County\nOwner: 3 - Town or Township Highway Agency\nMaintenance Responsibility: 3 - Town or Township Highway Agency\n\nRoute FAS 0123 /\nStructure #00010 / (Routine)\nTR 02 FAS 123 over MID. BR WILLIAMS RIVER\nTeam Lead: Jeremy Spooner, Inspection Date: 07/05/2024\nLocation: 0.3 MI S JCT. VT.11\n© OpenStreetMap contributors\n43.21230, -72.73812\n\nI\n(\n(\n(\nA\nB\nC\n*\n\nRoute FAS 0123 /\nStructure #00010 / (Routine)\nTR 02 FAS 123 over MID. BR WILLIAMS RIVER\nTeam Lead: Jeremy Spooner, Inspection Date: 07/05/2024\nIDENTIFICATION CLASSIFICATION\n(1) State Names 50 - Vermont (112) NBIS Bridge Length Y\n(8) Structure Number 200123001013232 (104) Highway System 0\n(5) Inventory Route 1 (26) Functional Class 7 - Rural Major Collector\n(2) Highway Agency District 2 - District 2 (100) Defense Highwa

In [9]:
print(tables)
len(tables)

[[[['INSPECTIONS *'], ['(90) Inspection Date 07/05/2024'], ['(91) Frequency 24'], ['(92) Critical Feature Inspection Done Freq. (Mon) Date'], ['A: Fracture Critical Detail No'], ['B: Underwater Inspection No'], ['C: Other Special Inspection'], ['* The inspection date and frequency information in this box contains\nthe current NBI date and frequency information. Please refer to the\nreport header for the date this inspection was conducted.']]], [[[None, None, None, None, None, '', '', ''], ['ELEMENTS', 'DESCRIPTION', 'UNITS', 'TOTAL', 'CS1', 'CS2', 'CS3', 'CS4'], ['38 RC Slab SF 884 834 25 25 0\n1080 Delamination/Spall/Patched Area SF 25 0 0 25 0\n1090 Exposed Rebar SF 25 0 25 0 0\n510 Wearing Surfaces SF 768 768 0 0 0', None, None, None, None, None, None, None], ['330 Metal Bridge Railing LF 50 25 25 0 0\n1020 Connection LF 25 0 25 0 0', None, None, None, None, None, None, None], ['804 Concrete Fascia LF 50 25 25 0 0\n1090 Exposed Rebar LF 25 0 25 0 0', None, None, None, None, None, No

5

### Clean Text

`re.sub()` this stands for subsititue 

In [10]:
text = re.sub(r'\s+', ' ', text_with_tables)  # Normalize whitespace
text = re.sub(r'(\d) - ', r'\1 - ', text_with_tables)
print(text[:400])

Route FAS 0123 /
Structure #00010 / (Routine)
TR 02 FAS 123 over MID. BR WILLIAMS RIVER
Team Lead: Jeremy Spooner, Inspection Date: 07/05/2024
Town: 262 - WINDHAM
District 2, 25 - WINDHAM County
Owner: 3 - Town or Township Highway Agency
Maintenance Responsibility: 3 - Town or Township Highway Agency

Route FAS 0123 /
Structure #00010 / (Routine)
TR 02 FAS 123 over MID. BR WILLIAMS RIVER
Team Lead


In [11]:
doc = nlp(text)
# doc

In [12]:
text2 = re.sub(r'\s+', ' ', text2)  # Normalize whitespace
text2 = re.sub(r'(\d) - ', r'\1 - ', text2)
print(text2[:400])

District 2, 25 - WINDHAM County Owner: 3 - Town or Township Highway Agency Town: 262 - WINDHAM Maintenance Responsibility: 3 - Town or Township Highway Agency Team Lead: Jeremy Spooner, Inspection Date: 07/05/2024 Structure #00010 / (Routine) Route FAS 0123 / TR 02 FAS 123 over MID. BR WILLIAMS RIVER © OpenStreetMap contributors Location: 0.3 MI S JCT. VT.11 43.21230, -72.73812 Team Lead: Jeremy S


In [13]:
doc2 = nlp(text2)
# doc2

In [14]:
doc

Route FAS 0123 /
Structure #00010 / (Routine)
TR 02 FAS 123 over MID. BR WILLIAMS RIVER
Team Lead: Jeremy Spooner, Inspection Date: 07/05/2024
Town: 262 - WINDHAM
District 2, 25 - WINDHAM County
Owner: 3 - Town or Township Highway Agency
Maintenance Responsibility: 3 - Town or Township Highway Agency

Route FAS 0123 /
Structure #00010 / (Routine)
TR 02 FAS 123 over MID. BR WILLIAMS RIVER
Team Lead: Jeremy Spooner, Inspection Date: 07/05/2024
Location: 0.3 MI S JCT. VT.11
© OpenStreetMap contributors
43.21230, -72.73812

I
(
(
(
A
B
C
*

Route FAS 0123 /
Structure #00010 / (Routine)
TR 02 FAS 123 over MID. BR WILLIAMS RIVER
Team Lead: Jeremy Spooner, Inspection Date: 07/05/2024
IDENTIFICATION CLASSIFICATION
(1) State Names 50 - Vermont (112) NBIS Bridge Length Y
(8) Structure Number 200123001013232 (104) Highway System 0
(5) Inventory Route 1 (26) Functional Class 7 - Rural Major Collector
(2) Highway Agency District 2 - District 2 (100) Defense Highway 0 - The inventory route is not
(3

In [15]:
pattern = f"Structure\s+#(\d+)"
matches = re.finditer(pattern, doc.text, re.IGNORECASE)
structure_numbers = []
for match in matches:
    if match.group(1) not in structure_numbers:
        structure_numbers.append(match.group(1))
bridge_id = structure_numbers[0] if len(structure_numbers) == 1 else "Unknown"
bridge_id

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\MK\AppData\Local\Temp\ipykernel_7200\608127002.py:1: SyntaxWarning: invalid escape sequence '\s'
  pattern = f"Structure\s+#(\d+)"


'00010'

## Find section boundaries

In [16]:
sentences = list(doc.sents) # this is how you get the sentences
sentences[0].start_char

0

In [74]:
# senteences = list(doc.sents)
lines = doc.text.split('\n')
section_boundaries = []

section_headers = [
        "deck", "approach", "superstructure", "substructure", 
        "channel", "general observation","channel profile",
    ]
for i, line in enumerate(lines):
    clean_line = line.strip().lower()
    if not clean_line:
        continue
    for header in section_headers:
        if clean_line == header:
            section_boundaries.append((i, header.upper()))
            break
section_boundaries.sort(key=lambda x: x[0])
print(section_boundaries)

[(168, 'DECK'), (189, 'APPROACH'), (220, 'SUPERSTRUCTURE'), (254, 'SUBSTRUCTURE'), (269, 'CHANNEL'), (276, 'GENERAL OBSERVATION'), (323, 'CHANNEL PROFILE')]


In [75]:
section= {}
sections = []

for i, (line_idx, header) in enumerate(section_boundaries):
    end_idx = section_boundaries[i + 1][0] if  i < len(section_boundaries) - 1 else len(lines)
    
    section = {
        "index": (line_idx, end_idx),
        "content_array": lines[line_idx:end_idx],
        "content": "\n".join(lines[line_idx:end_idx]),
        "header": header
    }
    sections.append(section)

In [76]:
print(sections[2]["content"])

Superstructure
ELEMENTS DESCRIPTION UNITS TOTAL CS1 CS2 CS3 CS4
59 - Superstructure (6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.)
isolated moderate spalling on the downstream fascia with reinforcement steel having section loss.
B.C.07 Bridge Bearings Condition Rating (NOT APPLICABLE - Component does not exist.)
B.C.14 NSTM Inspection Condition (NOT APPLICABLE - Component does not exist.)

I
(
(
(
A
B
C
*

							
E	D	U	T	C	C	C	C
3							
3							
8							

							
E	D	U	T	C	C	C	C

							
E	D	U	T	C	C	C	C
2							
8							

Route FAS 0123 /
Structure #00010 / (Routine)
TR 02 FAS 123 over MID. BR WILLIAMS RIVER
Team Lead: Jeremy Spooner, Inspection Date: 07/05/2024


In [77]:
print(sections[0]["content"])

Deck
ELEMENTS DESCRIPTION UNITS TOTAL CS1 CS2 CS3 CS4
38 RC Slab SF 884 834 25 25 0
1080 Delamination/Spall/Patched Area SF 25 0 0 25 0
1090 Exposed Rebar SF 25 0 25 0 0
510 Wearing Surfaces SF 768 768 0 0 0
330 Metal Bridge Railing LF 50 25 25 0 0
1020 Connection LF 25 0 25 0 0
804 Concrete Fascia LF 50 25 25 0 0
1090 Exposed Rebar LF 25 0 25 0 0
58 - Deck (6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.)
Downstream fascia has spalling around the rail post with loss of support to the post
200 - Existing Wearing Surface Depth (5.5)
A21 - Deck Wearing Surface Condition (Satisfactory)
Scattered minor to moderate cracking
A39 - Deck Fascia Condition (Poor)
Downstream fascia that has heavy spalling with exposed rebar and bolt for the bridge rail post are pretty much rusted
through
B.C.05 Bridge Railing Condition Rating (SATISFACTORY - Widespread minor or isolated moderate defects.)
Downstream post have spalling and corrosion along the anchor bolts
B.C.08 Bri

## Get table and description boundary

we will use units here because these are present for everyt row 

In [78]:
units_patterns = [
        r'\bLF\b',    # Linear Feet
        r'\bEA\b',    # Each
        r'\bSF\b',    # Square Feet
        r'\bCF\b',    # Cubic Feet
        r'\bFT\b',    # Feet
        r'\bIN\b',    # Inches
        r'\bM\b',     # Meters
        r'\bMM\b'     # Millimeters
    ]
units_regex = '|'.join(units_patterns)

In [79]:
units_regex

'\\bLF\\b|\\bEA\\b|\\bSF\\b|\\bCF\\b|\\bFT\\b|\\bIN\\b|\\bM\\b|\\bMM\\b'

A simple check to asses if content has a table or not

In [80]:
if "LF" in sections[0]["content"]:
    print("Yeah")
else:
    print("Nah")

Yeah


### Get last table line

In [81]:
doc_ = nlp(sections[0]["content"])
lines = doc_.text.split("\n")

last_table_line = -1
for idx, line in enumerate(lines):
    # print(re.search(units_regex, line))
    if re.search(units_regex, line):
        print(line)
        last_table_line = idx

38 RC Slab SF 884 834 25 25 0
1080 Delamination/Spall/Patched Area SF 25 0 0 25 0
1090 Exposed Rebar SF 25 0 25 0 0
510 Wearing Surfaces SF 768 768 0 0 0
330 Metal Bridge Railing LF 50 25 25 0 0
1020 Connection LF 25 0 25 0 0
804 Concrete Fascia LF 50 25 25 0 0
1090 Exposed Rebar LF 25 0 25 0 0


In [82]:
lines

['Deck',
 'ELEMENTS DESCRIPTION UNITS TOTAL CS1 CS2 CS3 CS4',
 '38 RC Slab SF 884 834 25 25 0',
 '1080 Delamination/Spall/Patched Area SF 25 0 0 25 0',
 '1090 Exposed Rebar SF 25 0 25 0 0',
 '510 Wearing Surfaces SF 768 768 0 0 0',
 '330 Metal Bridge Railing LF 50 25 25 0 0',
 '1020 Connection LF 25 0 25 0 0',
 '804 Concrete Fascia LF 50 25 25 0 0',
 '1090 Exposed Rebar LF 25 0 25 0 0',
 '58 - Deck (6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.)',
 'Downstream fascia has spalling around the rail post with loss of support to the post',
 '200 - Existing Wearing Surface Depth (5.5)',
 'A21 - Deck Wearing Surface Condition (Satisfactory)',
 'Scattered minor to moderate cracking',
 'A39 - Deck Fascia Condition (Poor)',
 'Downstream fascia that has heavy spalling with exposed rebar and bolt for the bridge rail post are pretty much rusted',
 'through',
 'B.C.05 Bridge Railing Condition Rating (SATISFACTORY - Widespread minor or isolated moderate defects.)',
 

In [83]:
lines[last_table_line]

'1090 Exposed Rebar LF 25 0 25 0 0'

In [84]:

table = lines[:last_table_line+1]
desc = lines[last_table_line+1:]
table, desc

(['Deck',
  'ELEMENTS DESCRIPTION UNITS TOTAL CS1 CS2 CS3 CS4',
  '38 RC Slab SF 884 834 25 25 0',
  '1080 Delamination/Spall/Patched Area SF 25 0 0 25 0',
  '1090 Exposed Rebar SF 25 0 25 0 0',
  '510 Wearing Surfaces SF 768 768 0 0 0',
  '330 Metal Bridge Railing LF 50 25 25 0 0',
  '1020 Connection LF 25 0 25 0 0',
  '804 Concrete Fascia LF 50 25 25 0 0',
  '1090 Exposed Rebar LF 25 0 25 0 0'],
 ['58 - Deck (6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.)',
  'Downstream fascia has spalling around the rail post with loss of support to the post',
  '200 - Existing Wearing Surface Depth (5.5)',
  'A21 - Deck Wearing Surface Condition (Satisfactory)',
  'Scattered minor to moderate cracking',
  'A39 - Deck Fascia Condition (Poor)',
  'Downstream fascia that has heavy spalling with exposed rebar and bolt for the bridge rail post are pretty much rusted',
  'through',
  'B.C.05 Bridge Railing Condition Rating (SATISFACTORY - Widespread minor or isolated mo

In [85]:
table_text = '\n'.join(table)
print(table_text)

Deck
ELEMENTS DESCRIPTION UNITS TOTAL CS1 CS2 CS3 CS4
38 RC Slab SF 884 834 25 25 0
1080 Delamination/Spall/Patched Area SF 25 0 0 25 0
1090 Exposed Rebar SF 25 0 25 0 0
510 Wearing Surfaces SF 768 768 0 0 0
330 Metal Bridge Railing LF 50 25 25 0 0
1020 Connection LF 25 0 25 0 0
804 Concrete Fascia LF 50 25 25 0 0
1090 Exposed Rebar LF 25 0 25 0 0


In [86]:
desc_text = '\n'.join(desc)
print(desc_text)

58 - Deck (6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.)
Downstream fascia has spalling around the rail post with loss of support to the post
200 - Existing Wearing Surface Depth (5.5)
A21 - Deck Wearing Surface Condition (Satisfactory)
Scattered minor to moderate cracking
A39 - Deck Fascia Condition (Poor)
Downstream fascia that has heavy spalling with exposed rebar and bolt for the bridge rail post are pretty much rusted
through
B.C.05 Bridge Railing Condition Rating (SATISFACTORY - Widespread minor or isolated moderate defects.)
Downstream post have spalling and corrosion along the anchor bolts
B.C.08 Bridge Joints Condition Rating (NOT APPLICABLE - Bridge does not have deck joints.)


## Get bridge components from tables

### Create component regex

In [92]:
processed_components =[]
bridge_components = [
    "Deck",
    "Girder/Beam",
    "Truss",
    "Arch",
    "Pier",
    "Abutment",
    "Bearing",
    "Pile",
    "Railing",
    "Pile Cap",
    "Bent",
    "Parapet/Guard Rail",
    "Expansion Joint",
    "Foundation",
    "Backwall",
    "Diaphragm",
    "Fascia",
    "Approach",
    "Slab",
    "Approach Slab",
    "Wingwall",
    "Spandrel",
    "Cantilever",
    "Counterweight",
    "Cutwater",
    "Flood Arches",
    "Suspension Cable",
    "Tower",
    "Cable-Stay",
    "Wing/Retaining Wall",
    "Pier Cap",
    "Bearing Pedestal",
    "Stringer",
    "Footing",
    "Shear Key",
    "Drainage System",
    "Seismic Device",
    "Crash Barrier",
    "Culvert",
    "Culvert Pipe",
    "Headwall",
    "Apron",
    "Invert",
    "Soffit",
    "Skewback",
    "Keystone",
    "Camber",
    "Tie Rod",
    "Dampener"
]
bridge_components = list(set(bridge_components))
materials = ["Reinforced Concrete", "Prestressed Concrete", "RC", "PC", "Steel", "Concrete", "Timber", "Wood", "Metal", "Metal Bridge"]


In [93]:
for component in bridge_components:
        # Handle slash-separated components
        if "/" in component:
            parts = [re.escape(part.strip()) for part in component.split("/")]
            # Create pattern that allows for typos or partial matches
            pattern = f"(?:{'|'.join(parts)}|{re.escape(component)})"
            
            # Special case for "Wing/Retaining Wall" - also match if "Wall" has a typo
            if "Wall" in component:
                wall_variations = ["Wall", "Wal", "Walls"]
                wall_pattern = f"(?:{'|'.join(wall_variations)})"
                # Replace "Wall" in the pattern with the variations
                pattern = pattern.replace(re.escape("Wall"), wall_pattern)
        else:
            pattern = re.escape(component)
        
        # Make trailing 's' optional for plurals (e.g. match "Piers" with "Pier")
        if pattern.endswith("s\\"):
            pattern = pattern[:-2] + "s?"
        else:
            pattern = pattern + "s?"
            
        # Remove number if present (e.g. "Abutment 1" -> "Abutment( \d+)?")
        if " " in pattern and any(c.isdigit() for c in pattern):
            pattern = re.sub(r'\\\ \d+', '', pattern) + r'(\ \d+)?'
            
        processed_components.append(pattern)

In [94]:
material_pattern = f"(?:{'|'.join(materials)})\\s+"
component_pattern = f"(?:{'|'.join(processed_components)})"
    
# Final pattern including optional material prefix and component
final_pattern = fr"(\d+\s+(?:{material_pattern})?{component_pattern})"

In [95]:
component_sections = []
doc_t = nlp(table_text)
    
matches = list(re.finditer(final_pattern, doc_t.text))
for i, match in enumerate(matches):
    start_pos = match.start()
        
    # Determine end position (either next component or end of text)
    if i < len(matches) - 1:
        end_pos = matches[i+1].start()
    else:
        end_pos = len(table_text)
        
    # Extract the full section text
    section_text = table_text[start_pos:end_pos].strip()
        
    # Extract just the component name (may include material)
    component_text = match.group(1)
        
    component_sections.append({
            "full_section": section_text,
            "component": component_text,
            "start": start_pos,
            "end": end_pos
        })

In [ ]:
print(component_sections[0]["full_section"])

38 RC Slab SF 884 834 25 25 0
1080 Delamination/Spall/Patched Area SF 25 0 0 25 0
1090 Exposed Rebar SF 25 0 25 0 0
510 Wearing Surfaces SF 768 768 0 0 0


In [97]:
print(component_sections[1]["full_section"])

330 Metal Bridge Railing LF 50 25 25 0 0
1020 Connection LF 25 0 25 0 0


In [98]:
print(component_sections[2]["full_section"])

804 Concrete Fascia LF 50 25 25 0 0
1090 Exposed Rebar LF 25 0 25 0 0


## Get Component and defect Info

### Component info

In [99]:
# Regex patterns for extracting data
id_pattern = r"^(\d+)\s+"
units_pattern = r"\b(LF|SF|EA|CF|FT|IN)\b"
values_pattern = r"\b(\d+(?:\s+\d+)*)\s*$"

In [100]:
def extract_component_values(component_text, full_section, component_data):
    """
    Extract component values directly after the component name and unit.
    
    Args:
        component_text (str): The component text (e.g. "215 Reinforced Concrete Abutment")
        full_section (str): The full text of the component section
        component_data (dict): The component data dictionary to update
    """
    # Escape special characters in the component text for regex
    escaped_component = re.escape(component_text)
    
    # Pattern to find the component's unit and values
    # Look for exactly 5 numbers after the unit (TOTAL + CS1-CS4)
    pattern = fr"{escaped_component}\s+(LF|SF|EA|CF|FT|IN)\s+(\d+\s+\d+\s+\d+\s+\d+\s+\d+)"
    match = re.search(pattern, full_section)
    
    if match:
        component_data["units"] = match.group(1)
        values_str = match.group(2)
        component_data["values"] = [int(val) for val in values_str.split()]
    else:
        # Fallback pattern for cases with fewer or more values
        pattern = fr"{escaped_component}\s+(LF|SF|EA|CF|FT|IN)\s+(\d+(?:\s+\d+)*?)(?:\s+\d{{3,4}}\s|$)"
        match = re.search(pattern, full_section)
        
        if match:
            component_data["units"] = match.group(1)
            values_str = match.group(2)
            component_data["values"] = [int(val) for val in values_str.split()]
        else:
            # Last resort fallback
            unit_pattern = r"\b(LF|SF|EA|CF|FT|IN)\s+(\d+(?:\s+\d+){0,4})"
            pos = full_section.find(component_text) + len(component_text)
            remaining_text = full_section[pos:pos+30]  # Look at limited text to avoid capturing too much
            unit_match = re.search(unit_pattern, remaining_text)
            
            if unit_match:
                component_data["units"] = unit_match.group(1)
                values_str = unit_match.group(2)

                component_data["values"] = [int(val) for val in values_str.split()]

In [101]:
def extract_defects(section_text, component_data):
    """
    Extract defect information from a component section.
    
    Args:
        section_text (str): The full component section text
        component_data (dict): The component data dictionary to update
    """
    # Pattern to match defect entries: ID + Description + Units + Values
    # Use a more specific pattern to match exactly the format we expect
    defect_pattern = r"(\d{3,4})\s+([^L][^F][^S][^E][^A][^\d]+?)\s+(LF|SF|EA|CF|FT|IN)\s+(\d+(?:\s+\d+){0,4})"
    
    # Find all defects in the section text
    defect_matches = re.finditer(defect_pattern, section_text)
    
    for match in defect_matches:
        defect_id = match.group(1)
        defect_desc = match.group(2).strip()
        defect_unit = match.group(3)
        defect_values_str = match.group(4)
        defect_values = [int(val) for val in defect_values_str.split()]
        
        # Skip if this is the component ID itself (first entry)
        if defect_id == component_data.get("id"):
            continue
            
        # Create defect entry
        defect = {
            "id": defect_id,
            "description": defect_desc,
            "unit": defect_unit,
            "total": defect_values[0] if defect_values else None,
            "cs_values": defect_values[1:] if len(defect_values) > 1 else []
        }
        
        component_data["defects"].append(defect)

In [102]:
structured_components = []
for section in component_sections:
    component_data = {
            "text": section["component"],
            "full_section": section["full_section"],
            "defects": []  # Will hold defect information
        }
        
    # Extract ID
    id_match = re.search(id_pattern, section["component"])
    if id_match:
        print(id_match.group(1))
        component_data["id"] = id_match.group(1)
    units_match = re.search(units_pattern, section["full_section"])
    if units_match:
        print(units_match.group(1))
        component_data["units"] = units_match.group(1)
    values_match = re.search(values_pattern, section["full_section"])
    extract_component_values(section["component"], section["full_section"], component_data)
    extract_defects(section["full_section"], component_data)
    structured_components.append(component_data)

38
SF
330
LF
804
LF


Expetected Output
`[{'text': '215 Reinforced Concrete Abutment',
  'full_section': '215 Reinforced Concrete Abutment LF 51 42 6 3 0\n1080 Delamination/Spall/Patched Area LF 3 0 0 3 0\n1090 Exposed Rebar LF 1 0 1 0 0\n1130 Cracking (RC and Other) LF 5 0 5 0 0',
  'defects': [{'id': '1080',
    'description': 'Delamination/Spall/Patched Area',
    'unit': 'LF',
    'total': 3,
    'cs_values': [0, 0, 3, 0]},
   {'id': '1090',
    'description': 'Exposed Rebar',
    'unit': 'LF',
    'total': 1,
    'cs_values': [0, 1, 0, 0]},
   {'id': '1130',
    'description': 'Cracking (RC and Other)',
    'unit': 'LF',
    'total': 5,
    'cs_values': [0, 5, 0, 0]}],
  'id': '215',
  'units': 'LF',
  'values': [51, 42, 6, 3, 0]},
 {'text': '800 Reinforced Concrete Wing',
  'full_section': '800 Reinforced Concrete Wing/Retaining Wall EA 4 0 4 0 0\n1120 Efflorescence/Rust Staining EA 2 0 2 0 0\n1130 Cracking (RC and Other) EA 2 0 2 0 0',
  'defects': [{'id': '1120',
    'description': 'Efflorescence/Rust Staining',
    'unit': 'EA',
    'total': 2,
    'cs_values': [0, 2, 0, 0]},
   {'id': '1130',
    'description': 'Cracking (RC and Other)',
    'unit': 'EA',
    'total': 2,
    'cs_values': [0, 2, 0, 0]}],
  'id': '800',
  'units': 'EA',
  'values': [4, 0, 4, 0, 0]}]`

In [103]:
structured_components

[{'text': '38 RC Slab',
  'full_section': '38 RC Slab SF 884 834 25 25 0\n1080 Delamination/Spall/Patched Area SF 25 0 0 25 0\n1090 Exposed Rebar SF 25 0 25 0 0\n510 Wearing Surfaces SF 768 768 0 0 0',
  'defects': [{'id': '1080',
    'description': 'Delamination/Spall/Patched Area',
    'unit': 'SF',
    'total': 25,
    'cs_values': [0, 0, 25, 0]},
   {'id': '1090',
    'description': 'Exposed Rebar',
    'unit': 'SF',
    'total': 25,
    'cs_values': [0, 25, 0, 0]},
   {'id': '510',
    'description': 'Wearing Surfaces',
    'unit': 'SF',
    'total': 768,
    'cs_values': [768, 0, 0, 0]}],
  'id': '38',
  'units': 'SF',
  'values': [884, 834, 25, 25, 0]},
 {'text': '330 Metal Bridge Railing',
  'full_section': '330 Metal Bridge Railing LF 50 25 25 0 0\n1020 Connection LF 25 0 25 0 0',
  'defects': [{'id': '1020',
    'description': 'Connection',
    'unit': 'LF',
    'total': 25,
    'cs_values': [0, 25, 0, 0]}],
  'id': '330',
  'units': 'LF',
  'values': [50, 25, 25, 0, 0]},
 {'

## Get info from description text

### get contexts

In [104]:
desc_text.split("\n")

['58 - Deck (6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.)',
 'Downstream fascia has spalling around the rail post with loss of support to the post',
 '200 - Existing Wearing Surface Depth (5.5)',
 'A21 - Deck Wearing Surface Condition (Satisfactory)',
 'Scattered minor to moderate cracking',
 'A39 - Deck Fascia Condition (Poor)',
 'Downstream fascia that has heavy spalling with exposed rebar and bolt for the bridge rail post are pretty much rusted',
 'through',
 'B.C.05 Bridge Railing Condition Rating (SATISFACTORY - Widespread minor or isolated moderate defects.)',
 'Downstream post have spalling and corrosion along the anchor bolts',
 'B.C.08 Bridge Joints Condition Rating (NOT APPLICABLE - Bridge does not have deck joints.)']

In [105]:
def parse_condition(condition_text):
    """
    Parse condition text to extract rating and assessment description.
    
    Args:
        condition_text (str): The condition text from the header
        
    Returns:
        tuple: (condition_rating, assessment_description)
    """
    if not condition_text:
        return None, None
    
    # Handle "N/A" or "Not Applicable" cases
    if condition_text.upper() in ["N/A", "NOT APPLICABLE"]:
        return "Not Applicable", None
    
    # Handle simple condition ratings without explanations
    if condition_text.upper() in ["SATISFACTORY", "GOOD", "FAIR", "POOR"]:
        return condition_text.upper(), None
    
    # Pattern for conditions with numeric codes: "6 - SATISFACTORY CONDITION - explanation"
    code_pattern = r'(?:(\d+)\s*-\s*)?([A-Z\s]+)(?:\s*-\s*(.+))?'
    match = re.match(code_pattern, condition_text)
    
    if match:
        code = match.group(1)  # Numeric code like "6"
        rating = match.group(2).strip()  # Rating like "SATISFACTORY CONDITION"
        assessment = match.group(3)  # Explanation text
        
        # Normalize rating text (remove "CONDITION" if present)
        rating = rating.replace("CONDITION", "").strip()
        
        return rating, assessment
    
    # Default case - return the whole text as the rating
    return condition_text, None

In [106]:
header_pattern = r'([A-Z]?\d+\s+-\s+[^()]+)(?:\s*\(([^)]+)\))?'
    
    # Find all headers in the text
header_matches = list(re.finditer(header_pattern, desc_text))
    
    # If no headers found, return the whole text as one section
if not header_matches:
    desc_sections =  [{"header": None, "condition": None, "content": text.strip()}]
    
    # Create sections by finding the text between headers
desc_sections = []
    
for i, match in enumerate(header_matches):
    header = match.group(1).strip()
    condition_text = match.group(2).strip() if match.group(2) else None
        
        # Parse the condition and assessment
    condition_rating, assessment = parse_condition(condition_text)
        
        # Find the start of the content (after the header and condition)
    content_start = match.end()
        
        # Find the end of the content (start of next header or end of text)
    if i < len(header_matches) - 1:
        content_end = header_matches[i + 1].start()
    else:
        content_end = len(text)
        
        # Extract the content
    content = desc_text[content_start:content_end].strip()
        
    desc_sections.append({
            "header": header,
            "condition_text": condition_text,
            "condition_rating": condition_rating,
            "assessment": assessment,
            "content": content
        })
desc_sections

[{'header': '58 - Deck',
  'condition_text': '6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.',
  'condition_rating': 'SATISFACTORY',
  'assessment': 'structural elements show some minor deterioration.',
  'content': 'Downstream fascia has spalling around the rail post with loss of support to the post'},
 {'header': '200 - Existing Wearing Surface Depth',
  'condition_text': '5.5',
  'condition_rating': '5.5',
  'assessment': None,
  'content': ''},
 {'header': 'A21 - Deck Wearing Surface Condition',
  'condition_text': 'Satisfactory',
  'condition_rating': 'SATISFACTORY',
  'assessment': None,
  'content': 'Scattered minor to moderate cracking'},
 {'header': 'A39 - Deck Fascia Condition',
  'condition_text': 'Poor',
  'condition_rating': 'POOR',
  'assessment': None,
  'content': 'Downstream fascia that has heavy spalling with exposed rebar and bolt for the bridge rail post are pretty much rusted\nthrough\nB.C.05 Bridge Railing Condition Rating (SATISFA

## Matching components and desciption

### Extract component name

remeber here we would neeed to extend to specoal cases

In [44]:
def extract_component_name(text):
    """
    Extract the core component name from the text, removing ID and material prefixes.
    
    Args:
        text (str): The component text (e.g., "215 Reinforced Concrete Abutment")
        
    Returns:
        str: The core component name (e.g., "Abutment")
    """
    # Remove leading ID number
    text = re.sub(r'^\d+\s+', '', text)
    
    # Remove common material prefixes
    for material in materials:
        text = text.replace(material, '').strip()

    # we will need to extend special cases
    
    # Handle special case for "Wing/Retaining Wall"
    if "Wing" in text and "Wall" not in text:
        text = "Wing/Retaining Wall"
    
    return text

In [45]:
print(structured_components[0]['text'])
extract_component_name(structured_components[0]['text'])

215 Reinforced Concrete Abutment


'Abutment'

### Generate component variations

what about lowercase with s and no s?


Rememer here we would dstill need to special cases

In [46]:
def generate_component_variations(component_name):
    """
    Generate variations of a component name for flexible matching.
    
    Args:
        component_name (str): The core component name
        
    Returns:
        list: List of possible variations of the component name
    """
    variations = [component_name]
    
    # Add lowercase version
    variations.append(component_name.lower())
    
    # Handle singular/plural variations
    if component_name.endswith('s'):
        variations.append(component_name[:-1])  # Remove 's'
    else:
        variations.append(component_name + 's')  # Add 's'
    
    # Handle variations with numbers
    variations.append(component_name + r' \d+')  # Component followed by number
    
    # Handle compound components with slashes
    if '/' in component_name:
        parts = [part.strip() for part in component_name.split('/')]
        variations.extend(parts)
        
        # Special case for "Wing/Retaining Wall"
        if "Wing" in component_name and "Wall" in component_name:
            variations.append("Wingwall")
    return variations

In [47]:
print(extract_component_name(structured_components[0]['text']))
generate_component_variations(extract_component_name(structured_components[0]['text']))

Abutment


['Abutment', 'abutment', 'Abutments', 'Abutment \\d+']

### Match comoponent name in headers

In [48]:
def find_in_headers(component_variations, description_sections):
    """
    Try to find component variations in the section headers.
    
    Args:
        component_variations (list): List of component name variations
        description_sections (list): List of section dictionaries
        
    Returns:
        dict or None: The matching section or None if no match found
    """
    for section in description_sections:
        header = section["header"]
        
        if not header:  # to check if a string exists in a larger string just do this 
            continue
            
        # Check if any variation appears in the header
        for variation in component_variations:
            if variation.lower() in header.lower():
                return section
    
    return None

In [49]:

find_in_headers(generate_component_variations(extract_component_name(structured_components[1]['text'])), desc_sections)

{'header': 'A77 - Retaining/Wingwall Condition',
 'condition_text': 'Satisfactory',
 'condition_rating': 'SATISFACTORY',
 'assessment': None,
 'content': 'Scaling along outlet end'}

### Find in content

Double check  what is going here there might be an issue

In [50]:
def find_in_content(component_variations, description_sections):
    """
    Try to find component variations in the section content.
    
    Args:
        component_variations (list): List of component name variations
        description_sections (list): List of section dictionaries
        
    Returns:
        dict or None: The matching section or None if no match found
    """
    for section in description_sections:
        content = section["content"]
        
        if not content:
            continue
            
        # Check if any variation appears in the content
        for variation in component_variations:
            # Create a pattern that matches the variation as a whole word
            pattern = r'\b' + variation + r'\b'
            if re.search(pattern, content, re.IGNORECASE):
                return section
                
            # For patterns with \d+, we need a special approach
            if r'\d+' in variation:
                base = variation.replace(r'\d+', '')
                pattern = r'\b' + base + r'\d+\b'
                if re.search(pattern, content, re.IGNORECASE):
                    return section
    
    return None

In [51]:
find_in_content(generate_component_variations(extract_component_name(structured_components[0]['text'])), desc_sections)

{'header': '60 - Substructure',
 'condition_text': '6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.',
 'condition_rating': 'SATISFACTORY',
 'assessment': 'structural elements show some minor deterioration.',
 'content': 'Abutment 1 has approx. 1/4" wide vertical crack with spalling around the drain holes isolated moderate spalling on\ndownstream\nabutment 2 has minor cracks, leaks and isolated moderate spalling on the downstream'}

In [52]:
updated_components = []

for component in structured_components:
        # Create a copy of the component data
        updated_component = component.copy()
        
        # Extract the component name without the ID and material prefixes
        component_name = extract_component_name(component["text"])
        component_variations = generate_component_variations(component_name)
        
        # Try to find a match in the section headers first
        header_match = find_in_headers(component_variations, desc_sections)
        
        if header_match:
            # If found in a header, add the section information
            updated_component["description_section"] = header_match
        else:
            # If not found in headers, try to find mentions in the content
            content_match = find_in_content(component_variations, desc_sections)
            
            if content_match:
                updated_component["description_section"] = content_match
        
        updated_components.append(updated_component)
updated_components

[{'text': '215 Reinforced Concrete Abutment',
  'full_section': '215 Reinforced Concrete Abutment LF 51 42 6 3 0\n1080 Delamination/Spall/Patched Area LF 3 0 0 3 0\n1090 Exposed Rebar LF 1 0 1 0 0\n1130 Cracking (RC and Other) LF 5 0 5 0 0',
  'defects': [{'id': '1080',
    'description': 'Delamination/Spall/Patched Area',
    'unit': 'LF',
    'total': 3,
    'cs_values': [0, 0, 3, 0]},
   {'id': '1090',
    'description': 'Exposed Rebar',
    'unit': 'LF',
    'total': 1,
    'cs_values': [0, 1, 0, 0]},
   {'id': '1130',
    'description': 'Cracking (RC and Other)',
    'unit': 'LF',
    'total': 5,
    'cs_values': [0, 5, 0, 0]}],
  'id': '215',
  'units': 'LF',
  'values': [51, 42, 6, 3, 0],
  'description_section': {'header': '60 - Substructure',
   'condition_text': '6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.',
   'condition_rating': 'SATISFACTORY',
   'assessment': 'structural elements show some minor deterioration.',
   'content': 'Abutment 

## Get maintenance reccomendations

In [53]:
doc4

Route FAS 0123 /  

Structure #00010 /  (Routine)

TR 02  FAS 123 over MID. BR WILLIAMS RIVER

Team Lead: Jeremy Spooner,  Inspection Date: 07/05/2024

Town: 262 - WINDHAM

District 2, 25 - WINDHAM County

Owner: 3 - Town or Township Highway Agency

Maintenance Responsibility: 3 - Town or Township Highway Agency


Route FAS 0123 /  

Structure #00010 /  (Routine)

TR 02  FAS 123 over MID. BR WILLIAMS RIVER

Team Lead: Jeremy Spooner,  Inspection Date: 07/05/2024

Location: 0.3 MI S JCT. VT.11

43.21230, -72.73812

© OpenStreetMap contributors


Route FAS 0123 /  

Structure #00010 /  (Routine)

TR 02  FAS 123 over MID. BR WILLIAMS RIVER

Team Lead: Jeremy Spooner,  Inspection Date: 07/05/2024

IDENTIFICATION

CLASSIFICATION

(1) State Names
(8) Structure Number
(5) Inventory Route
(2) Highway Agency District
(3) County Code
(4) Place Code
(6) Features Intersected
(7) Facility Carried
(9) Location
(11) Mile Point
(12) Base Highway Network
(13) LRS Inventory Rte & Subrte
(16) Latitude
(1

### Get maintenance section boundaries

In [54]:
def find_maintenance_sections(doc):
    """
    Find all "Maintenance Needs" sections in the document, from the header to "Remarks".
    
    Args:
        doc: spaCy Doc object
        
    Returns:
        list: List of section text strings
    """
    text = doc.text
    
    # Pattern to match "Maintenance Needs" header
    maintenance_pattern = r'Maintenance\s+Needs'
    
    # Pattern to match "Remarks" header
    remarks_pattern = r'Remarks'
    
    # Find all occurrences of "Maintenance Needs"
    maintenance_matches = list(re.finditer(maintenance_pattern, text, re.IGNORECASE))
    
    sections = []
    
    for i, match in enumerate(maintenance_matches):
        # Find the start of this maintenance section
        section_start = match.start()
        
        # Find the end of this maintenance section (next "Remarks" or next "Maintenance Needs" or end of text)
        # First look for "Remarks" after this match
        remarks_match = re.search(remarks_pattern, text[section_start:], re.IGNORECASE)
        if remarks_match:
            section_end = section_start + remarks_match.start()
        else:
            # If no "Remarks" found, check if there's another "Maintenance Needs"
            if i < len(maintenance_matches) - 1:
                section_end = maintenance_matches[i + 1].start()
            else:
                # If this is the last maintenance section, go to the end of the text
                section_end = len(text)
        
        # Extract the section text
        section_text = text[section_start:section_end].strip()
        sections.append(section_text)
    
    return sections

In [55]:
def find_maintenance_sections(doc):
    """
    Find all "Maintenance Needs" sections in the document, from the header to "Remarks".
    
    Args:
        doc: spaCy Doc object
        
    Returns:
        list: List of section text strings
    """
    text = doc.text
    
    # Pattern to match "Maintenance Needs" header
    maintenance_pattern = r'Maintenance\s+Needs'
    
    # Pattern to match "Remarks" header
    remarks_pattern = r'Remarks'
    
    # Find all occurrences of "Maintenance Needs"
    maintenance_matches = list(re.finditer(maintenance_pattern, text, re.IGNORECASE))
    
    sections = []
    
    for i, match in enumerate(maintenance_matches):
        # Find the start of this maintenance section
        section_start = match.start()
        
        # Find the end of this maintenance section (next "Remarks" or next "Maintenance Needs" or end of text)
        # First look for "Remarks" after this match
        remarks_match = re.search(remarks_pattern, text[section_start:], re.IGNORECASE)
        
        if remarks_match:
            section_end = section_start + remarks_match.start()
        else:
            # If no "Remarks" found, check if there's another "Maintenance Needs"
            if i < len(maintenance_matches) - 1:
                section_end = maintenance_matches[i + 1].start()
            else:
                # If this is the last maintenance section, go to the end of the text
                section_end = len(text)
        
        # Extract the section text
        section_text = text[section_start:section_end].strip()
        sections.append(section_text)
    
    return sections

In [56]:
maintenance_sections = find_maintenance_sections(doc4)
print(maintenance_sections[0])
print(maintenance_sections[1])

Maintenance Needs

Date Reported: 07/05/2022

Priority:

4 - Maintenance Finding - Next 
Inspection Cycle

Type of Work:

12 - Deck - Fascia and curb repair or 
reconstruction

Status:

Open

Component:

Deck

Deficiency Description

Downstream fascia needs to be reconstructed and new anchor bolts for bridge rail post need to be installed same time.
Maintenance Needs

Date Reported: 07/05/2022

Priority:

4 - Maintenance Finding - Next 
Inspection Cycle

Type of Work:

37 - Channel - Debris/Aggradation 
removal

Status:

Open

Component:

Channel

Deficiency Description

Heavy aggregate buildup along abutment 1 should be removed due to narrowing of the stream bed


### Parse maintenance section boundaries

In [57]:
def extract_maintenance_info(maintenance_sections: List[str]) -> List[Dict]:
    """
    Extract maintenance information from an array of text sections
    
    Args:
        maintenance_sections: List of text strings, each representing a maintenance section
        
    Returns:
        List of dictionaries containing extracted maintenance information
    """
    # List to store all processed maintenance sections
    processed_sections = []
    
    # Process each section
    for section_text in maintenance_sections:
        current_section = {}
        current_section["full_text"] = section_text
        # Extract date reported
        date_match = re.search(r"Date Reported:\s*(\d{1,2}/\d{1,2}/\d{4})", section_text)
        if date_match:
            current_section["date_reported"] = date_match.group(1).strip()
        
        # Extract priority - look for content between "Priority:" and the next field
        priority_match = re.search(r"Priority:(?:\s*\n)?(.*?)(?=\n\s*(?:Type of Work:|Status:|Component:))", section_text, re.DOTALL)
        if priority_match:
            priority_text = priority_match.group(1).strip()
            # Clean up multi-line text
            priority_text = re.sub(r'\s+', ' ', priority_text)
            current_section["priority"] = priority_text
        
        # Extract type of work - look for content between "Type of Work:" and the next field
        work_match = re.search(r"Type of Work:(?:\s*\n)?(.*?)(?=\n\s*(?:Status:|Component:|Deficiency))", section_text, re.DOTALL)
        if work_match:
            work_text = work_match.group(1).strip()
            # Clean up multi-line text
            work_text = re.sub(r'\s+', ' ', work_text)
            current_section["type_of_work"] = work_text
        
        # Extract status
        status_match = re.search(r"Status:(?:\s*\n)?(.*?)(?=\n\s*(?:Component:|Deficiency))", section_text, re.DOTALL)
        if status_match:
            status_text = status_match.group(1).strip()
            current_section["status"] = status_text
        
        # Extract component
        component_match = re.search(r"Component:(?:\s*\n)?(.*?)(?=\n\s*(?:Deficiency|Remarks))", section_text, re.DOTALL)
        if component_match:
            component_text = component_match.group(1).strip()
            current_section["component"] = component_text
        
        # Try multiple patterns for deficiency description, just like in the first code
        deficiency_patterns = [
            r"Deficiency Description\s*(.+?)(?=Remarks:|$)",
            r"Deficiency Description\n(.+?)(?=\n\n|\n[A-Z]|$)"
        ]
        
        for pattern in deficiency_patterns:
            deficiency_match = re.search(pattern, section_text, re.DOTALL | re.IGNORECASE)
            if deficiency_match:
                deficiency_text = deficiency_match.group(1).strip()
                # Clean up multi-line text
                deficiency_text = re.sub(r'\s+', ' ', deficiency_text)
                current_section["deficiency_description"] = deficiency_text
                break  # Found a match, no need to try other patterns
        
        # Only add sections that have at least some data
        if current_section:
            processed_sections.append(current_section)
    
    return processed_sections

In [58]:
print(maintenance_sections[0])

Maintenance Needs

Date Reported: 07/05/2022

Priority:

4 - Maintenance Finding - Next 
Inspection Cycle

Type of Work:

12 - Deck - Fascia and curb repair or 
reconstruction

Status:

Open

Component:

Deck

Deficiency Description

Downstream fascia needs to be reconstructed and new anchor bolts for bridge rail post need to be installed same time.


In [59]:
type(maintenance_sections[0])

str

In [60]:
maintenance_sections

['Maintenance Needs\n\nDate Reported: 07/05/2022\n\nPriority:\n\n4 - Maintenance Finding - Next \nInspection Cycle\n\nType of Work:\n\n12 - Deck - Fascia and curb repair or \nreconstruction\n\nStatus:\n\nOpen\n\nComponent:\n\nDeck\n\nDeficiency Description\n\nDownstream fascia needs to be reconstructed and new anchor bolts for bridge rail post need to be installed same time.',
 'Maintenance Needs\n\nDate Reported: 07/05/2022\n\nPriority:\n\n4 - Maintenance Finding - Next \nInspection Cycle\n\nType of Work:\n\n37 - Channel - Debris/Aggradation \nremoval\n\nStatus:\n\nOpen\n\nComponent:\n\nChannel\n\nDeficiency Description\n\nHeavy aggregate buildup along abutment 1 should be removed due to narrowing of the stream bed']

In [63]:
mainntenance_needs = extract_maintenance_info(maintenance_sections)
mainntenance_needs

[{'full_text': 'Maintenance Needs\n\nDate Reported: 07/05/2022\n\nPriority:\n\n4 - Maintenance Finding - Next \nInspection Cycle\n\nType of Work:\n\n12 - Deck - Fascia and curb repair or \nreconstruction\n\nStatus:\n\nOpen\n\nComponent:\n\nDeck\n\nDeficiency Description\n\nDownstream fascia needs to be reconstructed and new anchor bolts for bridge rail post need to be installed same time.',
  'date_reported': '07/05/2022',
  'priority': '4 - Maintenance Finding - Next Inspection Cycle',
  'type_of_work': '12 - Deck - Fascia and curb repair or reconstruction',
  'status': 'Open',
  'component': 'Deck',
  'deficiency_description': 'Downstream fascia needs to be reconstructed and new anchor bolts for bridge rail post need to be installed same time.'},
 {'full_text': 'Maintenance Needs\n\nDate Reported: 07/05/2022\n\nPriority:\n\n4 - Maintenance Finding - Next \nInspection Cycle\n\nType of Work:\n\n37 - Channel - Debris/Aggradation \nremoval\n\nStatus:\n\nOpen\n\nComponent:\n\nChannel\n\nD

## Component and maintenance needs matching

In [62]:
updated_components_new = []
for component in updated_components:
        updated_component = component.copy()
        updated_component["maintenance_needs"] = []
        updated_components_new.append(updated_component)
updated_components_new

[{'text': '215 Reinforced Concrete Abutment',
  'full_section': '215 Reinforced Concrete Abutment LF 51 42 6 3 0\n1080 Delamination/Spall/Patched Area LF 3 0 0 3 0\n1090 Exposed Rebar LF 1 0 1 0 0\n1130 Cracking (RC and Other) LF 5 0 5 0 0',
  'defects': [{'id': '1080',
    'description': 'Delamination/Spall/Patched Area',
    'unit': 'LF',
    'total': 3,
    'cs_values': [0, 0, 3, 0]},
   {'id': '1090',
    'description': 'Exposed Rebar',
    'unit': 'LF',
    'total': 1,
    'cs_values': [0, 1, 0, 0]},
   {'id': '1130',
    'description': 'Cracking (RC and Other)',
    'unit': 'LF',
    'total': 5,
    'cs_values': [0, 5, 0, 0]}],
  'id': '215',
  'units': 'LF',
  'values': [51, 42, 6, 3, 0],
  'description_section': {'header': '60 - Substructure',
   'condition_text': '6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.',
   'condition_rating': 'SATISFACTORY',
   'assessment': 'structural elements show some minor deterioration.',
   'content': 'Abutment 

In [64]:
def match_by_component_name(need, components):
    """
    Match a maintenance need to a component by direct component name comparison.
    
    Args:
        need (dict): Maintenance need dictionary
        components (list): List of component dictionaries
        
    Returns:
        dict or None: Matched component or None if no match found
    """
    if not need.get("component"):
        return None
    
    need_component = need["component"].lower()
    
    for component in components:
        # Extract clean component name
        clean_name = extract_component_name(component["text"]).lower()
        
        # Check for direct match or substring match
        if (clean_name == need_component or 
            clean_name in need_component or 
            need_component in clean_name):
            return component
        
        # Handle special cases
        if clean_name == "wing/retaining wall" and ("wing" in need_component or "retaining" in need_component or "wall" in need_component):
            return component
        
        if clean_name == "abutment" and "abutment" in need_component:
            return component
    
    return None

In [65]:
def match_by_description(need, components):
    """
    Match a maintenance need to a component by finding component mentions in the description.
    
    Args:
        need (dict): Maintenance need dictionary
        components (list): List of component dictionaries
        
    Returns:
        dict or None: Matched component or None if no match found
    """
    if not need.get("deficiency_description"):
        return None
    
    description = need["deficiency_description"].lower()
    
    # Try each component
    for component in components:
        # Extract clean component name and variations
        clean_name = extract_component_name(component["text"])
        variations = generate_component_variations(clean_name)
        
        # Check if any variation appears in the description
        for variation in variations:
            pattern = r'\b' + variation.lower() + r'\b'
            if re.search(pattern, description):
                return component
            
            # Handle numbered components like "abutment 1"
            if variation.lower() + r' \d+' in variations:
                pattern = r'\b' + variation.lower() + r' \d+\b'
                if re.search(pattern, description):
                    return component
    
    return None


In [66]:
# Process each maintenance need
for need in mainntenance_needs:
        # Try to match by direct component name
    matched_component = match_by_component_name(need, updated_components_new)
        
        # If no match, try to match by mentions in the deficiency description
    if not matched_component:
        matched_component = match_by_description(need, updated_components_new)
        
        # If a match was found, add the maintenance need to the component
    if matched_component:
        matched_component["maintenance_needs"].append(need)
    
updated_components_new

[{'text': '215 Reinforced Concrete Abutment',
  'full_section': '215 Reinforced Concrete Abutment LF 51 42 6 3 0\n1080 Delamination/Spall/Patched Area LF 3 0 0 3 0\n1090 Exposed Rebar LF 1 0 1 0 0\n1130 Cracking (RC and Other) LF 5 0 5 0 0',
  'defects': [{'id': '1080',
    'description': 'Delamination/Spall/Patched Area',
    'unit': 'LF',
    'total': 3,
    'cs_values': [0, 0, 3, 0]},
   {'id': '1090',
    'description': 'Exposed Rebar',
    'unit': 'LF',
    'total': 1,
    'cs_values': [0, 1, 0, 0]},
   {'id': '1130',
    'description': 'Cracking (RC and Other)',
    'unit': 'LF',
    'total': 5,
    'cs_values': [0, 5, 0, 0]}],
  'id': '215',
  'units': 'LF',
  'values': [51, 42, 6, 3, 0],
  'description_section': {'header': '60 - Substructure',
   'condition_text': '6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.',
   'condition_rating': 'SATISFACTORY',
   'assessment': 'structural elements show some minor deterioration.',
   'content': 'Abutment 

In [67]:
len(updated_components_new)

2

In [ ]:
import re


def match_by_component_name(need, components):
    """
    Match a maintenance need to a component by direct component name comparison.
    
    Args:
        need (dict): Maintenance need dictionary
        components (list): List of component dictionaries
        
    Returns:
        dict or None: Matched component or None if no match found
    """
    if not need.get("component"):
        return None
    
    need_component = need["component"].lower()
    
    for component in components:
        # Extract clean component name
        clean_name = extract_component_name(component["text"]).lower()
        
        # Check for direct match or substring match
        if (clean_name == need_component or 
            clean_name in need_component or 
            need_component in clean_name):
            return component
        
        # Handle special cases
        if clean_name == "wing/retaining wall" and ("wing" in need_component or "retaining" in need_component or "wall" in need_component):
            return component
        
        if clean_name == "abutment" and "abutment" in need_component:
            return component
    
    return None

def match_by_description(need, components):
    """
    Match a maintenance need to a component by finding component mentions in the description.
    
    Args:
        need (dict): Maintenance need dictionary
        components (list): List of component dictionaries
        
    Returns:
        dict or None: Matched component or None if no match found
    """
    if not need.get("deficiency_description"):
        return None
    
    description = need["deficiency_description"].lower()
    
    # Try each component
    for component in components:
        # Extract clean component name and variations
        clean_name = extract_component_name(component["text"])
        variations = generate_component_variations(clean_name)
        
        # Check if any variation appears in the description
        for variation in variations:
            pattern = r'\b' + variation.lower() + r'\b'
            if re.search(pattern, description):
                return component
            
            # Handle numbered components like "abutment 1"
            if variation.lower() + r' \d+' in variations:
                pattern = r'\b' + variation.lower() + r' \d+\b'
                if re.search(pattern, description):
                    return component
    
    return None



def generate_component_variations(component_name):
    """
    Generate variations of a component name for flexible matching.
    
    Args:
        component_name (str): The core component name
        
    Returns:
        list: List of possible variations of the component name
    """
    variations = [component_name]
    
    # Add lowercase version
    variations.append(component_name.lower())
    
    # Handle singular/plural variations
    if component_name.endswith('s'):
        variations.append(component_name[:-1])  # Remove 's'
    else:
        variations.append(component_name + 's')  # Add 's'
    
    # Handle variations with numbers
    variations.append(component_name + r' \d+')  # Component followed by number
    
    # Handle compound components with slashes
    if '/' in component_name:
        parts = [part.strip() for part in component_name.split('/')]
        variations.extend(parts)
        
        # Special case for "Wing/Retaining Wall"
        if "Wing" in component_name and "Wall" in component_name:
            variations.append("Wingwall")
    
    return variations

def format_components_with_maintenance(components_with_maintenance):
    """
    Format the components with their matched maintenance needs for readable output.
    
    Args:
        components_with_maintenance (list): List of components with matched maintenance needs
        
    Returns:
        str: Formatted output text
    """
    result = []
    
    for i, component in enumerate(components_with_maintenance):
        result.append(f"COMPONENT {i+1}")
        result.append(f"ID: {component.get('id', 'Unknown')}")
        result.append(f"Name: {component['text']}")
        
        if "units" in component:
            result.append(f"Units: {component['units']}")
            
        if "values" in component:
            result.append(f"Values: {component['values']}")
        
        # Display linked maintenance needs
        if "maintenance_needs" in component and component["maintenance_needs"]:
            result.append("\nMaintenance Needs:")
            for j, need in enumerate(component["maintenance_needs"]):
                result.append(f"\n  NEED {j+1}:")
                result.append(f"  Date Reported: {need.get('date_reported', 'Unknown')}")
                result.append(f"  Priority: {need.get('priority', 'Unknown')}")
                result.append(f"  Status: {need.get('status', 'Unknown')}")
                result.append(f"  Type of Work: {need.get('type_of_work', 'Unknown')}")
                result.append(f"  Component: {need.get('component', 'Unknown')}")
                result.append(f"  Description: {need.get('deficiency_description', 'Unknown')}")
        else:
            result.append("\nNo maintenance needs found for this component.")
        
        # Display defects
        if "defects" in component and component["defects"]:
            result.append("\nDefects:")
            for defect in component["defects"]:
                result.append(f"  • {defect['id']} - {defect['description']}")
                result.append(f"    Unit: {defect['unit']}")
                result.append(f"    Total: {defect['total']}")
                result.append(f"    CS Values: {defect['cs_values']}")
        
        result.append("\n" + "-" * 50 + "\n")
    
    return "\n".join(result)

# Example usage
if __name__ == "__main__":
    # Sample maintenance needs data
    maintenance_needs = [
        {
            'date_reported': '07/05/2022',
            'priority': '4 - Maintenance Finding - Next Inspection Cycle',
            'type_of_work': '12 - Deck - Fascia and curb repair or reconstruction', 
            'status': 'Open',
            'component': 'Deck',
            'deficiency_description': 'Downstream fascia needs to be reconstructed and new anchor bolts for bridge rail post need to be installed same time.'
        },
        {
            'date_reported': '07/05/2022',
            'priority': '4 - Maintenance Finding - Next Inspection Cycle',
            'type_of_work': '37 - Channel - Debris/Aggradation removal',
            'status': 'Open',
            'component': 'Channel',
            'deficiency_description': 'Heavy aggregate buildup along abutment 1 should be removed due to narrowing of the stream bed'
        }
    ]
    
    # Sample component data
    components = [
        {
            'text': '215 Reinforced Concrete Abutment',
            'full_section': '215 Reinforced Concrete Abutment LF 51 42 6 3 0\n1080 Delamination/Spall/Patched Area LF 3 0 0 3 0\n1090 Exposed Rebar LF 1 0 1 0 0\n1130 Cracking (RC and Other) LF 5 0 5 0 0',
            'defects': [
                {'id': '1080', 'description': 'Delamination/Spall/Patched Area', 'unit': 'LF', 'total': 3, 'cs_values': [0, 0, 3, 0]},
                {'id': '1090', 'description': 'Exposed Rebar', 'unit': 'LF', 'total': 1, 'cs_values': [0, 1, 0, 0]},
                {'id': '1130', 'description': 'Cracking (RC and Other)', 'unit': 'LF', 'total': 5, 'cs_values': [0, 5, 0, 0]}
            ],
            'id': '215',
            'units': 'LF',
            'values': [51, 42, 6, 3, 0],
            'description_section': {
                'header': '60 - Substructure',
                'condition_text': '6 - SATISFACTORY CONDITION - structural elements show some minor deterioration.',
                'condition_rating': 'SATISFACTORY',
                'assessment': 'structural elements show some minor deterioration.',
                'content': 'Abutment 1 has approx. 1/4" wide vertical crack with spalling around the drain holes isolated moderate spalling on\ndownstream\nabutment 2 has minor cracks, leaks and isolated moderate spalling on the downstream'
            }
        },
        {
            'text': '800 Reinforced Concrete Wing/Retaining Wall',
            'full_section': '800 Reinforced Concrete Wing/Retaining Wall EA 4 0 4 0 0\n1120 Efflorescence/Rust Staining EA 2 0 2 0 0\n1130 Cracking (RC and Other) EA 2 0 2 0 0',
            'defects': [
                {'id': '1120', 'description': 'Efflorescence/Rust Staining', 'unit': 'EA', 'total': 2, 'cs_values': [0, 2, 0, 0]},
                {'id': '1130', 'description': 'Cracking (RC and Other)', 'unit': 'EA', 'total': 2, 'cs_values': [0, 2, 0, 0]}
            ],
            'id': '800',
            'units': 'EA',
            'values': [4, 0, 4, 0, 0],
            'description_section': {
                'header': 'A77 - Retaining/Wingwall Condition',
                'condition_text': 'Satisfactory',
                'condition_rating': 'SATISFACTORY',
                'assessment': None,
                'content': 'Scaling along outlet end'
            }
        }
    ]
    
    # Match maintenance needs to components
    components_with_maintenance = match_maintenance_to_components(maintenance_needs, components)
    
    # Format and print the results
    formatted_output = format_components_with_maintenance(components_with_maintenance)
    print(formatted_output)